In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from visualization_helper import *

In [ ]:
import numpy as np
import numpy.linalg as la

In [ ]:
input_path = '../../data/sphere_cap_0.3.json'
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
import mesh
target_mesh = mesh.Mesh(input_data['target_v'], input_data['target_f'])


### Initialization

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
thickness = io.material_params[6]
curr_um.targetDeploymentHeight = thickness

In [ ]:
curr_um.energy()

### Visualization

In [ ]:
rod_colors = get_color_field(curr_um, input_data)

In [ ]:
import mesh
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

### Hessian Validation

In [ ]:
curr_um.updateSourceFrame()

In [ ]:
curr_um.hessianSparsityPattern()

In [ ]:
curr_um.hessian()

In [ ]:
import umbrella_finite_diff

In [ ]:
import importlib
importlib.reload(umbrella_finite_diff)

In [ ]:
perturbation = np.random.uniform(-1e-3, 1e-3, curr_um.numDoF())

In [ ]:
dof = curr_um.getDoFs()

In [ ]:
import copy

In [ ]:
curr_um.setDoFs(dof + perturbation)
# curr_um.setDoFs(perturbation_along_segment_dof)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 0.1
curr_um.attractionWeight = 0.1
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0)
# curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.Constant
curr_um.repulsionEnergyWeight = 0.1

In [ ]:
curr_um.targetDeploymentHeight = thickness

In [ ]:
curr_um.updateSourceFrame()

In [ ]:
umbrella_finite_diff.hessian_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Repulsion, etype=elastic_rods.EnergyType.Full, maxStepSize=1e-1)

In [ ]:
umbrella_finite_diff.hessian_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Deployment, etype=elastic_rods.EnergyType.Full, maxStepSize=1e-1)

In [ ]:
umbrella_finite_diff.hessian_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Attraction, etype=elastic_rods.EnergyType.Full, maxStepSize=1e-1)

In [ ]:
umbrella_finite_diff.hessian_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Elastic, etype=elastic_rods.EnergyType.Full)

In [ ]:
var_types = ['FULL', 'RodSegment', 'Joint']
var_indices = {'FULL': range(0, curr_um.numDoF()),
               'RodSegment': range(0, curr_um.dofOffsetForJoint(0)),
               'Joint': range( curr_um.dofOffsetForJoint(0), curr_um.numDoF())}

In [ ]:
umbrella_finite_diff.hessian_convergence_block_plot(curr_um, var_types, var_indices, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Full, etype=elastic_rods.EnergyType.Full)

In [ ]:
position_indices = [np.arange(curr_um.dofOffsetForJoint(i), curr_um.dofOffsetForJoint(i) + 3) for i in range(curr_um.numJoints())]
position_indices = np.array(position_indices).flatten()

omega_indices = [np.arange(curr_um.dofOffsetForJoint(i) + 3, curr_um.dofOffsetForJoint(i) + 6) for i in range(curr_um.numJoints())]
omega_indices = np.array(omega_indices).flatten()

alpha_indices = [np.arange(curr_um.dofOffsetForJoint(i) + 6, curr_um.dofOffsetForJoint(i) + 7) for i in range(curr_um.numJoints())]
alpha_indices = np.array(alpha_indices).flatten()

length_indices = []
for i in range(curr_um.numJoints()):
    length_indices += list(np.arange(curr_um.dofOffsetForJoint(i) + 7, curr_um.dofOffsetForJoint(i) + curr_um.joint(i).numDoF()))
length_indices = np.array(length_indices)

In [ ]:
var_types = ['position', 'omega', 'alpha', 'length']
var_indices = {'position': position_indices,
               'omega': omega_indices,
               'alpha': alpha_indices,
               'length': length_indices}

In [ ]:
umbrella_finite_diff.hessian_convergence_block_plot(curr_um, var_types, var_indices, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Full, etype=elastic_rods.EnergyType.Full)